## Примеры использования meduza

In [1]:
#!pip install meduza

In [1]:
import meduza

In [2]:
url = "https://meduza.io/paragraph/2022/02/11/hochu-stat-graficheskim-dizaynerom-no-s-risunkom-u-menya-ne-ochen-kak-byt-maksimalno-korotkoe-ob-yasnenie"

article = meduza.get(url)

article["title"]

article["second_title"] = "Trump and Putin meet in Helsinki, the ‘Deep State’ consensus, and Pussy Riot's heart-to-heart with the police"

In [3]:
article["tag"]["name"] == 'партнерский материал'

True

In [4]:
meduza.RU_TAGS

['новости',
 'истории',
 'разбор',
 'шапито',
 'игры',
 'подкасты',
 'партнерский материал']

## Попытка достать ссылки из датасета meduza.csv.gz

Полезная, но пока не использованная ссылка: https://habr.com/ru/articles/283058/

In [1]:
from corus import load_ods_meduza
from tqdm import tqdm
import numpy as np
import pickle

In [6]:
path = "meduza.csv.gz"
records = list(load_ods_meduza(path))

In [7]:
urls = [record.url for record in records]
texts = [record.text for record in records]

In [8]:
len(urls)

71806

In [9]:
texts[0]

'С\xa0марта 2019 года российским военным запрещено пользоваться смартфонами и\xa0публиковать в\xa0интернете фотографии со\xa0службы. Соответствующие поправки к\xa0закону «О\xa0статусе военнослужащих» правительство подготовило в\xa0сентябре 2018 года, а\xa0Госдума и\xa0Совет Федерации одобрили их\xa0в\xa0феврале 2019-го. В\xa0марте закон подписал президент России Владимир Путин, и\xa017\xa0марта онnвступилnв\xa0силу. Российских военных сразу\xa0же начали наказывать за\xa0найденные у\xa0них смартфоны и\xa0фото в\xa0социальных сетях: обычно им\xa0дают от\xa0пяти до\xa015 суток ареста, выяснила «Медуза», изучив решения военных судов.nВ\xa0пояснительной записке к\xa0поправкам правительствоnсвязывалоnих\xa0необходимость с\xa0военной операцией в\xa0Сирии: еще до\xa0того, как Россия в\xa0сентябре 2015 года официально объявила о\xa0ее\xa0начале, расследователи заявлялиnо\xa0присутствии в\xa0странеnроссийских военных\xa0— в\xa0том числе на\xa0основании их\xa0фотографий из\xa0соцсетей. Ранее учас

In [10]:
responses = []

In [11]:
batch_size = 10

In [12]:
len(urls) // batch_size

7180

1590 батч пропущен

In [ ]:
for i in range(1872, len(urls) // batch_size):
    temp = []
    for url, text in tqdm(zip(urls[batch_size * i: batch_size * (i + 1)], 
                        texts[batch_size * i: batch_size * (i + 1)])):
        resp = meduza.get(url)
        temp.append({"url": url, "text": text,"json": resp})
    if len(temp) == batch_size:
        responses += temp
    else:
        print("problem", i)
        break

In [25]:
import concurrent.futures

In [26]:
def fetch_url(url, text):
    resp = meduza.get(url)
    return {"url": url, "text": text, "json": resp}

In [216]:
for i in range(7116 + 1,  len(urls) // batch_size):
    temp = []
    urls_iter = urls[batch_size * i:batch_size * (i+1)]
    texts_iter = texts[batch_size * i:batch_size * (i+1)]
    with concurrent.futures.ThreadPoolExecutor(max_workers=6) as executor:
        futures = [executor.submit(fetch_url, url, text) for url, text in zip(urls_iter, texts_iter)]
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(urls_iter)):
            temp.append(future.result())
    responses += temp

100%|██████████| 10/10 [00:01<00:00,  8.72it/s]


In [217]:
len(responses)

71790

In [218]:
len(np.unique([res["url"] for res in responses]))

71790

In [219]:
tags = [resp["json"]["tag"]["name"] for resp in responses]

In [220]:
list(zip(*np.unique(tags, return_counts=True)))

[('игры', 643),
 ('истории', 9955),
 ('картотека', 47),
 ('медуза', 5),
 ('новости', 49794),
 ('партнерский материал', 1177),
 ('подкасты', 400),
 ('полигон', 468),
 ('разбор', 1206),
 ('реклама', 5),
 ('спецпроект', 84),
 ('тест', 2),
 ('шапито', 8001),
 ('шторм', 3)]

In [221]:
with open("meduza_response1.pkl", "wb") as f:
    pickle.dump(responses, f)

In [224]:
with open("meduza_response1.pkl", "rb") as f:
    loaded_response = pickle.load(f)
len(loaded_response)

71790

In [211]:
loaded_response[-1]["json"]["tag"]

{'name': 'новости'}

In [134]:
responses = loaded_response

## Крафтим датасет

In [2]:
with open("meduza_response1.pkl", "rb") as f:
    loaded_response = pickle.load(f)
len(loaded_response)

71790

In [3]:
tags = [res["json"]["tag"]["name"] for res in loaded_response]

In [6]:
all_dataset = dict(list(zip(*np.unique(tags, return_counts=True))))

{'игры': 643,
 'истории': 9955,
 'картотека': 47,
 'медуза': 5,
 'новости': 49794,
 'партнерский материал': 1177,
 'подкасты': 400,
 'полигон': 468,
 'разбор': 1206,
 'реклама': 5,
 'спецпроект': 84,
 'тест': 2,
 'шапито': 8001,
 'шторм': 3}

In [7]:
big_dataset_dist = {'игры': 643,
 'истории': 9955,
 'новости': 49794,
 'партнерский материал': 1177,
 'подкасты': 400,
 'полигон': 468,
 'разбор': 1206,
 'шапито': 8001}

In [248]:
# small_dataset_dist = {'игры': 100, 'истории': 100, 'новости': 1000, 
        #    'партнерский материал': 500, 'подкасты': 100, 
        #    'разбор': 100, 'шапито': 100}

In [11]:
import pandas as pd
import numpy as np
from dataclasses import make_dataclass

Text = make_dataclass("Text", [("title", str), ("text", str), ("tag", str), ("url", str)])

In [12]:
np.random.seed(5050)
indexes = []
for tag, size in big_dataset_dist.items():
    indexes += list(np.random.choice([i for i, t in enumerate(tags) if t == tag], size=size, replace=False))

In [13]:
texts = []
sliced_responses = [loaded_response[i] for i in indexes]
for resp in sliced_responses:
    texts.append(Text(np.nan, resp["text"], resp["json"]["tag"]["name"], resp["url"]))


In [16]:
dataset = pd.DataFrame(texts)
dataset = dataset[dataset["text"] != '']

In [19]:
dict(list(zip(*np.unique(dataset["tag"], return_counts=True))))

{'игры': 53,
 'истории': 9847,
 'новости': 49746,
 'партнерский материал': 752,
 'подкасты': 400,
 'полигон': 312,
 'разбор': 1203,
 'шапито': 7973}

In [20]:
dataset.to_csv("meduza_dataset_big.csv", index=False)